## Predicting images using imagnet VGG16 weights
This code is aggrigated from Keras' main developer.  His github is https://github.com/fchollet.  His github includes lots of code and examples.  His blog is also a good read.

### V

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing import image
from keras.models import model_from_json
from keras import backend as K
from keras import applications
import numpy as np
import json
from keras.utils.data_utils import get_file

In [3]:
model = applications.vgg16.VGG16(include_top=True, weights='imagenet', input_tensor=None, input_shape=None)

In [14]:
CLASS_INDEX = None
CLASS_INDEX_PATH = 'https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json'


def preprocess_input(x, dim_ordering='default'):
    if dim_ordering == 'default':
        dim_ordering = K.image_dim_ordering()
    assert dim_ordering in {'tf', 'th'}

    if dim_ordering == 'th':
        x[:, 0, :, :] -= 103.939
        x[:, 1, :, :] -= 116.779
        x[:, 2, :, :] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, ::-1, :, :]
    else:
        x[:, :, :, 0] -= 103.939
        x[:, :, :, 1] -= 116.779
        x[:, :, :, 2] -= 123.68
        # 'RGB'->'BGR'
        x = x[:, :, :, ::-1]
    return x


def decode_predictions(preds, top=5):
    global CLASS_INDEX
    if len(preds.shape) != 2 or preds.shape[1] != 1000:
        raise ValueError('`decode_predictions` expects '
                         'a batch of predictions '
                         '(i.e. a 2D array of shape (samples, 1000)). '
                         'Found array with shape: ' + str(preds.shape))
    if CLASS_INDEX is None:
        fpath = get_file('imagenet_class_index.json',
                         CLASS_INDEX_PATH,
                         cache_subdir='models')
        CLASS_INDEX = json.load(open(fpath))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        result = [tuple(CLASS_INDEX[str(i)]) + (pred[i],) for i in top_indices]
        results.append(result)
    return results

In [26]:
img_path = 'drogo.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

('Input image shape:', (1, 224, 224, 3))


In [27]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds))

('Predicted:', [[(u'n02105412', u'kelpie', 0.26755354), (u'n02107312', u'miniature_pinscher', 0.18241876), (u'n02107142', u'Doberman', 0.11221209), (u'n02104365', u'schipperke', 0.053030066), (u'n03124170', u'cowboy_hat', 0.050872561)]])
